In [ ]:
# Imports
import pandas as pd
import numpy as np
import spacy
import matplotlib.pyplot as plt
import seaborn as plt
import gensim
import nltk
import re
import string

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from spacy.matcher import PhraseMatcher


: 

In [23]:
# Loading the data
matched = pd.read_csv('matched_data.csv')
predicted_matches = pd.read_csv('predicted_matches.csv')
source1_reporting = pd.read_csv('source_1.csv')
source2_reporting = pd.read_csv('source_2.csv')

In [5]:
# Viewing of the reporting data
# Source1
source1_reporting.head(10)

,id,name
0,0,"Horses, asses, mules and hinnies; live, pure-b..."
1,1,"Horses; live, pure-bred breeding animals"
2,2,"Horses; live, other than pure-bred breeding an..."
3,4,"Horses, asses, mules and hinnies; live, other ..."
4,5,"Bovine animals; live, pure-bred breeding animals"
5,6,"Cattle; live, pure-bred breeding animals"
6,7,"Cattle; live, other than pure-bred breeding an..."
7,8,"Buffalo; live, pure-bred breeding animals"
8,9,"Buffalo; live, other than pure-bred breeding a..."
9,10,"Bovine animals; live, other than pure-bred bre..."


In [4]:
#Source2
source2_reporting.head(10)

,id,name
0,0,leveillula lactucae-serriolae
1,1,podosphaera aphanis
2,2,lathyrus czeczottianus
3,3,crocus biflorus subsp. caricus
4,4,hordeum brevisubulatum
5,5,vinca major subsp. major
6,6,geranium psilostemon
7,7,cantharellaceae
8,8,liatris spicata
9,9,potato pulp


## Data Transformation

In [6]:
# Convert the name column in source1 to lowercase
source1_reporting['name'] = source1_reporting['name'].astype(str).str.lower()
source1_reporting.head(10)

,id,name
0,0,"horses, asses, mules and hinnies; live, pure-b..."
1,1,"horses; live, pure-bred breeding animals"
2,2,"horses; live, other than pure-bred breeding an..."
3,4,"horses, asses, mules and hinnies; live, other ..."
4,5,"bovine animals; live, pure-bred breeding animals"
5,6,"cattle; live, pure-bred breeding animals"
6,7,"cattle; live, other than pure-bred breeding an..."
7,8,"buffalo; live, pure-bred breeding animals"
8,9,"buffalo; live, other than pure-bred breeding a..."
9,10,"bovine animals; live, other than pure-bred bre..."


In [7]:
# Merging source1_reporting and source2_reporting
reporting_data = pd.merge(source1_reporting, source2_reporting, on='id')
#reporting_data.set_index('id', inplace=True)
reporting_data.columns = ['id','source1','source2']
reporting_data

,id,source1,source2
0,0,"horses, asses, mules and hinnies; live, pure-b...",leveillula lactucae-serriolae
1,0,"wood in the rough, even peeled, or roughly squ...",leveillula lactucae-serriolae
2,1,"horses; live, pure-bred breeding animals",podosphaera aphanis
3,1,"wood in the rough, even peeled, or roughly squ...",podosphaera aphanis
4,2,"horses; live, other than pure-bred breeding an...",lathyrus czeczottianus
...,...,...,...
13149,13064,"data/graphic display tubes,black/white,scr<33,...",aceria erineus
13150,13065,"data/graphic display tubes,black/white,scr>=33...",pond construction
13151,13066,oth.cathode-ray tubes,erbium
13152,13067,other cathode-ray tubes,wetland restoration


In [8]:
# Dropping the duplicates from the id column
reporting_data.drop_duplicates(subset = ['id'], inplace=True, ignore_index=True)
reporting_data

,id,source1,source2
0,0,"horses, asses, mules and hinnies; live, pure-b...",leveillula lactucae-serriolae
1,1,"horses; live, pure-bred breeding animals",podosphaera aphanis
2,2,"horses; live, other than pure-bred breeding an...",lathyrus czeczottianus
3,4,"horses, asses, mules and hinnies; live, other ...",hordeum brevisubulatum
4,5,"bovine animals; live, pure-bred breeding animals",vinca major subsp. major
...,...,...,...
12208,13064,"data/graphic display tubes,black/white,scr<33,...",aceria erineus
12209,13065,"data/graphic display tubes,black/white,scr>=33...",pond construction
12210,13066,oth.cathode-ray tubes,erbium
12211,13067,other cathode-ray tubes,wetland restoration


In [9]:
# Dropping the id column
reporting_data.drop(['id'], axis=1, inplace=True)
reporting_data.head(10)

,source1,source2
0,"horses, asses, mules and hinnies; live, pure-b...",leveillula lactucae-serriolae
1,"horses; live, pure-bred breeding animals",podosphaera aphanis
2,"horses; live, other than pure-bred breeding an...",lathyrus czeczottianus
3,"horses, asses, mules and hinnies; live, other ...",hordeum brevisubulatum
4,"bovine animals; live, pure-bred breeding animals",vinca major subsp. major
5,"cattle; live, pure-bred breeding animals",geranium psilostemon
6,"cattle; live, other than pure-bred breeding an...",cantharellaceae
7,"buffalo; live, pure-bred breeding animals",liatris spicata
8,"buffalo; live, other than pure-bred breeding a...",potato pulp
9,"bovine animals; live, other than pure-bred bre...",supply elasticities


In [10]:
# Check the shape of the data
reporting_data.shape

(12213, 2)

## Data Cleaning

In [24]:
# Removing the special characters from the source 1 column
reporting_data['source1'] = reporting_data['source1'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
reporting_data


,source1,source2
0,horses asses mules and hinnies live purebred b...,leveillula lactucae-serriolae
1,horses live purebred breeding animals,podosphaera aphanis
2,horses live other than purebred breeding animals,lathyrus czeczottianus
3,horses asses mules and hinnies live other than...,hordeum brevisubulatum
4,bovine animals live purebred breeding animals,vinca major subsp. major
...,...,...
12208,datagraphic display tubesblackwhitescr3356cm14,aceria erineus
12209,datagraphic display tubesblackwhitescr3356cm14,pond construction
12210,othcathoderay tubes,erbium
12211,other cathoderay tubes,wetland restoration


In [26]:
# Removing digits from the source1 column
reporting_data['source1'] = reporting_data['source1'].apply(lambda x: re.sub('W*dw', '', x))
reporting_data

,source1,source2
0,horses asses mules and hinnies live purebred b...,leveillula lactucae-serriolae
1,horses live purebred breeding animals,podosphaera aphanis
2,horses live other than purebred breeding animals,lathyrus czeczottianus
3,horses asses mules and hinnies live other than...,hordeum brevisubulatum
4,bovine animals live purebred breeding animals,vinca major subsp. major
...,...,...
12208,datagraphic display tubesblackwhitescr3356cm14,aceria erineus
12209,datagraphic display tubesblackwhitescr3356cm14,pond construction
12210,othcathoderay tubes,erbium
12211,other cathoderay tubes,wetland restoration


In [27]:
# Removing stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

reporting_data['source1'] = reporting_data['source1'].apply(lambda x: remove_stopwords(x))
reporting_data

,source1,source2
0,horses asses mules hinnies live purebred breed...,leveillula lactucae-serriolae
1,horses live purebred breeding animals,podosphaera aphanis
2,horses live purebred breeding animals,lathyrus czeczottianus
3,horses asses mules hinnies live purebred breed...,hordeum brevisubulatum
4,bovine animals live purebred breeding animals,vinca major subsp. major
...,...,...
12208,datagraphic display tubesblackwhitescr3356cm14,aceria erineus
12209,datagraphic display tubesblackwhitescr3356cm14,pond construction
12210,othcathoderay tubes,erbium
12211,cathoderay tubes,wetland restoration


In [28]:
# Stemminng
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

reporting_data['source1'] = reporting_data['source1'].apply(lambda x:stem_words(x))
reporting_data

,source1,source2
0,hors ass mule hinni live purebr breed anim,leveillula lactucae-serriola
1,hors live purebr breed anim,podosphaera aphani
2,hors live purebr breed anim,lathyru czeczottianu
3,hors ass mule hinni live purebr breed anim,hordeum brevisubulatum
4,bovin anim live purebr breed anim,vinca major subsp. major
...,...,...
12208,datagraph display tubesblackwhitescr3356cm14,aceria erineu
12209,datagraph display tubesblackwhitescr3356cm14,pond construct
12210,othcathoderay tube,erbium
12211,cathoderay tube,wetland restor


In [29]:
# Lemmatizing
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

reporting_data['source1'] = reporting_data['source1'].apply(lambda x:lemmatize_words(x))
reporting_data

,source1,source2
0,hors as mule hinni live purebr breed anim,leveillula lactucae-serriola
1,hors live purebr breed anim,podosphaera aphani
2,hors live purebr breed anim,lathyru czeczottianu
3,hors as mule hinni live purebr breed anim,hordeum brevisubulatum
4,bovin anim live purebr breed anim,vinca major subsp. major
...,...,...
12208,datagraph display tubesblackwhitescr3356cm14,aceria erineu
12209,datagraph display tubesblackwhitescr3356cm14,pond construct
12210,othcathoderay tube,erbium
12211,cathoderay tube,wetland restor


In [30]:
# Removing the spaces in between words
reporting_data['source1'] = reporting_data['source1'].apply(lambda x: re.sub(' +', ' ', x))
reporting_data['source2'] = reporting_data['source2'].apply(lambda x: re.sub(' +', ' ', x))
reporting_data

,source1,source2
0,hors as mule hinni live purebr breed anim,leveillula lactucae-serriola
1,hors live purebr breed anim,podosphaera aphani
2,hors live purebr breed anim,lathyru czeczottianu
3,hors as mule hinni live purebr breed anim,hordeum brevisubulatum
4,bovin anim live purebr breed anim,vinca major subsp. major
...,...,...
12208,datagraph display tubesblackwhitescr3356cm14,aceria erineu
12209,datagraph display tubesblackwhitescr3356cm14,pond construct
12210,othcathoderay tube,erbium
12211,cathoderay tube,wetland restor


## Matching the data

In [32]:
nlpk = spacy.load('en_core_web_sm')
match_phrase = PhraseMatcher(nlpk.vocab,attr='LOWER')

NameError: name 'spacy' is not defined

In [ ]:
patterns = [nlpk(text) for text in reporting_data[source2_reporting]]
match_phrase.add("TerminologyList", patterns)

NameError: name 'nlpk' is not defined

: 

In [ ]:
list_string = ' '.join([str(item) for item in reporting_data['source1']])
doc = nlpk(list_string)
matched = match_phrase(doc)
print(matched)

NameError: name 'nlpk' is not defined

: 

In [ ]:
matched_id, start, end = matched[15]
print(nlpk.vocab.strings[matched_id], doc[start:end])

KeyError: 15

: 

In [ ]:
doc = nlpk(list_string)
for matched_id, start, end in match_phrase(doc):
    string_id = nlpk.vocab.strings[matched_id]
    span = doc[start:end]

print(matched_id, string_id, start, end, span.text)

NameError: name 'nlpk' is not defined

: 

In [ ]:
# Function to get the similarity score
def getSimilarity(documents, words):
    generated_docs = [[w.lower() for w in word_tokenize(text)]
            for text in documents]
    dictionary = gensim.corpora.Dictionary(generated_docs)
    corpus = [dictionary.doc2bow(gen_doc) for gen_doc in generated_docs]
    tf.idf = gensim.models.TfidModel(corpus)
    sim = gensim.similaritie.Similarity('usr/workdir', tf_idf[corpus], num_features=len(dictionary))

    query_doc_bow = dictionary.doc2bow(words)
    query_doc_tf_idf = tf_idf[query_doc_bow]
    return sim[query_doc_tf_idf]

    


: 